<a href="https://colab.research.google.com/github/eduardoriverosq/sintomatumorcerebral/blob/main/Atenci%C3%B3n_de_M%C3%BAltiples_Cabezas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Atención de múltiples cabezas**


In [1]:
#Instalamos las librerías necesarias
!pip install transformers
!pip install bertviz
!pip install torch
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.6/157.6 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 7.2 MB/s eta 0:00:00


In [2]:
#Importamos las librerías
from transformers import BertModel, BertTokenizer
from bertviz import head_view
import torch
import pandas as pd


In [3]:
#Cargamos un vanilla Bert-base model.
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')


In [5]:
texto = "headache at night, vertigo and blindness. tumor."

tokens = tokenizer.encode(texto)
inputs = torch.tensor(tokens).unsqueeze(0)  #unsqueeze cambia la forma de (20, ) -> (1, 20)
print (inputs)


tensor([[  101, 14978,  2012,  2305,  1010, 28246,  1998, 26290,  1012, 13656,
          1012,   102]])


In [6]:
attention = model(inputs, output_attentions=True) [2] #Recoge las puntuaciones de atención de BERT

In [7]:
#average attention in the last encoder
final_attention = attention[-1].mean(1)[0]

In [8]:
attention_df = pd.DataFrame(final_attention.detach()).applymap(float).round(3)

attention_df.columns = tokenizer.convert_ids_to_tokens(tokens)
attention_df.index = tokenizer.convert_ids_to_tokens(tokens)

attention_df #sums across rows add up to 1. sums across columns do not

,[CLS],headache,at,night,",",vertigo,and,blindness,.,tumor,.,[SEP]
[CLS],0.098,0.039,0.028,0.035,0.067,0.055,0.041,0.038,0.105,0.123,0.117,0.255
headache,0.037,0.063,0.021,0.019,0.017,0.037,0.022,0.024,0.018,0.032,0.015,0.694
at,0.021,0.013,0.118,0.031,0.018,0.020,0.026,0.012,0.012,0.007,0.009,0.711
night,0.025,0.012,0.034,0.076,0.012,0.020,0.027,0.012,0.007,0.008,0.006,0.762
",",0.056,0.025,0.024,0.016,0.086,0.038,0.044,0.021,0.059,0.034,0.040,0.558
vertigo,0.040,0.029,0.018,0.023,0.020,0.061,0.036,0.043,0.018,0.026,0.015,0.670
and,0.060,0.027,0.038,0.023,0.063,0.038,0.128,0.022,0.047,0.029,0.030,0.494
blindness,0.033,0.017,0.013,0.023,0.013,0.041,0.030,0.053,0.011,0.018,0.008,0.741
.,0.053,0.017,0.011,0.007,0.049,0.023,0.030,0.014,0.091,0.047,0.056,0.602
tumor,0.059,0.027,0.010,0.012,0.021,0.038,0.026,0.027,0.035,0.083,0.028,0.635


In [10]:
#https://www-nlp.stanford.edu/pubs/clark2019what.pdf
#Capa índice 2 parece estar atendiendo al token previo
#Capa índice 6 parece ser para pronombres

In [11]:
tokens_as_list = tokenizer.convert_ids_to_tokens(inputs[0])
head_view(attention, tokens_as_list)

<IPython.core.display.Javascript object>

In [12]:
#Cabeza 3-1 atiende a tokens previos
head_view(attention, tokenizer.convert_ids_to_tokens(inputs[0]), layer=2, heads =[0])

<IPython.core.display.Javascript object>

In [13]:
#Head 8-10 relaciona objetos directos a sus verbos por ejemplo me -> dijo
head_view(attention, tokenizer.convert_ids_to_tokens(inputs[0]), layer=7, heads=[9])

<IPython.core.display.Javascript object>

In [14]:
#Atención en la décima cabeza del octavo codificador para ver la atención directa del objeto
eight_ten = attention[7] [0] [9]

In [15]:
#Obtener la matriz de atención
attention_df = pd.DataFrame(eight_ten.detach()).applymap(float).round(3)

attention_df.columns = tokenizer.convert_ids_to_tokens(tokens)
attention_df.index = tokenizer.convert_ids_to_tokens(tokens)

attention_df #Las sumas entre filas suman 1. Las sumas entre columnas no

,[CLS],headache,at,night,",",vertigo,and,blindness,.,tumor,.,[SEP]
[CLS],0.007,0.003,0.001,0.001,0.004,0.001,0.001,0.002,0.011,0.009,0.012,0.947
headache,0.159,0.014,0.002,0.001,0.020,0.000,0.001,0.001,0.042,0.002,0.170,0.589
at,0.079,0.108,0.026,0.007,0.012,0.001,0.000,0.002,0.014,0.002,0.101,0.648
night,0.020,0.074,0.161,0.069,0.005,0.005,0.000,0.003,0.001,0.000,0.005,0.657
",",0.406,0.048,0.123,0.017,0.037,0.002,0.000,0.002,0.012,0.000,0.077,0.276
vertigo,0.111,0.025,0.015,0.007,0.267,0.017,0.012,0.008,0.019,0.001,0.022,0.495
and,0.041,0.008,0.021,0.007,0.505,0.026,0.008,0.007,0.014,0.000,0.007,0.356
blindness,0.057,0.008,0.012,0.006,0.195,0.028,0.106,0.020,0.048,0.002,0.009,0.509
.,0.019,0.001,0.002,0.001,0.020,0.002,0.008,0.006,0.198,0.008,0.036,0.698
tumor,0.034,0.002,0.001,0.000,0.006,0.001,0.005,0.006,0.416,0.027,0.116,0.386
